In [1]:
import tqdm
import numpy as np
import utils

# API Setup

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

# Dataset

We've abstracted away the code from the previous notebooks to focus on the concepts from this notebook.

In [3]:
data_index = 38
data = utils.load_data(sample_size=100)

Repo card metadata block was not found. Setting CardData to empty.


# Preprocessing
We've abstracted away the code from the previous notebooks to focus on the concepts from this notebook.

In [4]:
documents = utils.preprocess_data(data)

100%|██████████| 100/100 [00:00<00:00, 3232.90it/s]


# Chunking

We first demonstrate basic fixed size chunking with overlaps via LlamaIndex's [TokenTextSplitter](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/token_text_splitter/).

However, for the rest of the demo, we use a mix of two chunking methods - fixed-size chunking and content-aware chunking. Specifically, we use LlamaIndex's [SentenceSplitter](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/sentence_splitter/) which attempts to keep sentences and paragraphs together while maintaining chunks of roughly equal size.

We use a chunk size of 512 which is the maximum size that our embedding model (chosen further down) accepts. Within these chunks, we set an overlap of 20 tokens which corresponds to roughly 1-2 sentences.

You can examine other chunking methods implemented in LlamaIndex [here](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/).

### Fixed Size Chunking

In [5]:
from llama_index.core.node_parser import TokenTextSplitter
chunker = TokenTextSplitter(chunk_size=512, chunk_overlap=20)

In [6]:
sample = data["page"][data_index]
chunked = chunker.split_text(sample)
print(f"Document has ~{len(sample.split(' '))} words and was split into {len(chunked)} chunks of maximum 512 tokens.")

Document has ~10294 words and was split into 25 chunks of maximum 512 tokens.


In [7]:
# Can result in sentences being cut off in the middle
print(chunked[0])

= Michael Jordan = 
 
 Michael Jeffrey Jordan ( born February 17 , 1963 ) , also known by his initials , MJ , is an American retired professional basketball player . He is also a businessman , and principal owner and chairman of the Charlotte Hornets . Jordan played 15 seasons in the National Basketball Association ( NBA ) for the Chicago Bulls and Washington Wizards . His biography on the NBA website states : " By acclamation , Michael Jordan is the greatest basketball player of all time . " Jordan was one of the most effectively marketed athletes of his generation and was considered instrumental in popularizing the NBA around the world in the 1980s and 1990s . 
 Jordan played three seasons for coach Dean Smith at the University of North Carolina . He was a member of the Tar Heels ' national championship team in 1982 . Jordan joined the NBA 's Chicago Bulls in 1984 as the third overall draft pick . He quickly emerged as a league star , entertaining crowds with his prolific scoring . H

In [8]:
print(chunked[1])

1988 NBA Defensive Player of the Year Award . Among his numerous accomplishments , Jordan holds the NBA records for highest career regular season scoring average ( 30 @.@ 12 points per game ) and highest career playoff scoring average ( 33 @.@ 45 points per game ) . In 1999 , he was named the greatest North American athlete of the 20th century by ESPN , and was second to Babe Ruth on the Associated Press 's list of athletes of the century . Jordan is a two @-@ time inductee into the Basketball Hall of Fame , having been enshrined in 2009 for his individual career , and again in 2010 as part of the group induction of the 1992 United States men 's Olympic basketball team ( " The Dream Team " ) . He became a member of the FIBA Hall of Fame in 2015 . 
 Jordan is also known for his product endorsements . He fueled the success of Nike 's Air Jordan sneakers , which were introduced in 1985 and remain popular today . Jordan also starred in the 1996 feature film Space Jam as himself . In 2006 ,

### Content Aware Chunking

In [9]:
from llama_index.core.node_parser import SentenceSplitter
chunker = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [10]:
sample = data["page"][data_index]
chunked = chunker.split_text(sample)
print(f"Document has {len(sample.split(' '))} words and was split into {len(chunked)} chunks of maximum 512 tokens.")

Document has 10294 words and was split into 26 chunks of maximum 512 tokens.


In [11]:
# Does not cut sentences off in the middle
print(chunked[0])

= Michael Jordan = 
 
 Michael Jeffrey Jordan ( born February 17 , 1963 ) , also known by his initials , MJ , is an American retired professional basketball player . He is also a businessman , and principal owner and chairman of the Charlotte Hornets . Jordan played 15 seasons in the National Basketball Association ( NBA ) for the Chicago Bulls and Washington Wizards . His biography on the NBA website states : " By acclamation , Michael Jordan is the greatest basketball player of all time . " Jordan was one of the most effectively marketed athletes of his generation and was considered instrumental in popularizing the NBA around the world in the 1980s and 1990s . 
 Jordan played three seasons for coach Dean Smith at the University of North Carolina . He was a member of the Tar Heels ' national championship team in 1982 . Jordan joined the NBA 's Chicago Bulls in 1984 as the third overall draft pick . He quickly emerged as a league star , entertaining crowds with his prolific scoring . H

In [12]:
print(chunked[1])

Jordan 's individual accolades and accomplishments include five Most Valuable Player ( MVP ) Awards , ten All @-@ NBA First Team designations , nine All @-@ Defensive First Team honors , fourteen NBA All @-@ Star Game appearances , three All @-@ Star Game MVP Awards , ten scoring titles , three steals titles , six NBA Finals MVP Awards , and the 1988 NBA Defensive Player of the Year Award . Among his numerous accomplishments , Jordan holds the NBA records for highest career regular season scoring average ( 30 @.@ 12 points per game ) and highest career playoff scoring average ( 33 @.@ 45 points per game ) . In 1999 , he was named the greatest North American athlete of the 20th century by ESPN , and was second to Babe Ruth on the Associated Press 's list of athletes of the century . Jordan is a two @-@ time inductee into the Basketball Hall of Fame , having been enshrined in 2009 for his individual career , and again in 2010 as part of the group induction of the 1992 United States men '

### Perform Chunking on Dataset

Now we perform chunking on our entire dataset. We use the chunker's in-built function to do this on our list of documents. We get a list of [TextNodes](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_nodes/) as output. 

In [13]:
nodes = chunker.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
print(f"Documents before chunking: {len(documents)}")
print(f"Documents after chunking: {len(nodes)}")
nodes[0]

Documents before chunking: 100
Documents after chunking: 852


TextNode(id_='c7f5f02f-12b7-48d0-b64f-bcff596476ef', embedding=None, metadata={'title': 'Valkyria Chronicles III'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='18f9ce5d-0421-412b-bad4-064c6809e6e0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'Valkyria Chronicles III'}, hash='97c375d8ad713d727b964c7dd41b019777797b46662a06bc24ec8ccc392f6af2'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='d239d345-c077-455e-96bd-ce4e5343dc59', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='94bdb8573c0fb81586bf08bff93c5ee404e82ac21c627a13dd97b798040aa604')}, text='= Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in